
[Stack Exchange Data Dump](https://archive.org/details/stackexchange) by Stack Exchange Inc., available under [CC-BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/) license

Download and extract relevant XML files from https://archive.org/details/stackexchange

* https://archive.org/download/stackexchange/stackoverflow.com-Badges.7z
* https://archive.org/download/stackexchange/stackoverflow.com-Comments.7z
* https://archive.org/download/stackexchange/stackoverflow.com-PostLinks.7z
* https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z
* https://archive.org/download/stackexchange/stackoverflow.com-Tags.7z
* https://archive.org/download/stackexchange/stackoverflow.com-Users.7z
* https://archive.org/download/stackexchange/stackoverflow.com-Votes.7z

In [0]:
%sh

apt-get update
apt-get install -y xml-twig-tools p7zip-full

In [0]:
%sh

urls=(
    "https://archive.org/download/stackexchange/stackoverflow.com-Badges.7z"
    "https://archive.org/download/stackexchange/stackoverflow.com-Comments.7z"
    "https://archive.org/download/stackexchange/stackoverflow.com-PostLinks.7z"
    "https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z"
    "https://archive.org/download/stackexchange/stackoverflow.com-Tags.7z"
    "https://archive.org/download/stackexchange/stackoverflow.com-Users.7z"
    "https://archive.org/download/stackexchange/stackoverflow.com-Votes.7z"
)

# Choose a directory which has enough disk space - ideally some (mounted) cloud storage
cd /dbfs/tmp

for url in "${urls[@]}"; do
    filename=$(basename "$url")
    
    echo "Downloading $filename..."
    curl -L -O "$url"
    
    echo "Extracting $filename..."
    7z x -y "$filename"
    
    rm "$filename"
done

files_to_split=(
  "PostLinks.xml"
  "Badges.xml"
  "Comments.xml"
  "Users.xml"
  "Votes.xml"
  "Posts.xml"
)

# Split files into chunks of size 100MB
for filename in "${files_to_split[@]}"; do
    echo "Splitting $filename..."
    xml_split -s 100Mb -n 3 "$filename"
    
    rm "$filename"
done

In [0]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

# Data directory containing (splitted) XML files
data_dir = "/tmp"

In [0]:
tags_schema = StructType([
    StructField("Id", LongType(), False),
    StructField("TagName", StringType(), False),
    StructField("Count", IntegerType(), False),
    StructField("ExcerptPostId", LongType(), True),
    StructField("WikiPostId", LongType(), True)
])

tags = (
    spark.read
    .format("xml")
    .options(rootTag="tags", rowTag="row", attributePrefix="")
    .load(f"{data_dir}/Tags.xml", schema=tags_schema)
)

(
  tags
  .write
  # .format("parquet")
  .mode("overwrite")
  .save(f"{data_dir}/tags.parquet")
)

In [0]:
post_links_schema = StructType([
    StructField("Id", LongType(), False),
    StructField("CreationDate", TimestampType(), False),
    StructField("PostId", LongType(), False),
    StructField("RelatedPostId", LongType(), False),
    StructField("LinkTypeId", ShortType(), False)
])

post_links = (
    spark.read
    .format("xml")
    .options(rootTag="xml_split:root", rowTag="row", attributePrefix="")
    .load(f"{data_dir}/PostLinks-*.xml", schema=post_links_schema)
)

(
  post_links
  .write
  # .format("parquet")
  .mode("overwrite")
  .save(f"{data_dir}/post_links.parquet")
)

In [0]:
badges_schema = StructType([
    StructField("Id", LongType(), False),
    StructField("UserId", LongType(), False),
    StructField("Name", StringType(), False),
    StructField("Date", TimestampType(), False),
    StructField("Class", ShortType(), False),
    StructField("TagBased", BooleanType(), False)
])

badges = (
    spark.read
    .format("xml")
    .options(rootTag="xml_split:root", rowTag="row", attributePrefix="")
    .load(f"{data_dir}/Badges-*.xml", schema=badges_schema)
)

(
  badges
  .withColumn("Year", F.year("Date"))
  .write
  # .format("parquet")
  .partitionBy("Year")
  .mode("overwrite")
  .save(f"{data_dir}/badges.parquet")
)

In [0]:
comments_schema = StructType([
    StructField("Id", LongType(), False),
    StructField("PostId", LongType(), False),
    StructField("Score", IntegerType(), False),
    StructField("Text", StringType(), False),
    StructField("CreationDate", TimestampType(), False),
    StructField("UserId", LongType(), True),
    StructField("UserDisplayName", StringType(), True),
    StructField("ContentLicense", StringType(), False)
])

comments = (
    spark.read
    .format("xml")
    .options(rootTag="xml_split:root", rowTag="row", attributePrefix="")
    .load(f"{data_dir}/Comments-*.xml", schema=comments_schema)
)

(
  comments
  .withColumn("Year", F.year("CreationDate"))
  .write
  # .format("parquet")
  .partitionBy("Year")
  .mode("overwrite")
  .save(f"{data_dir}/comments.parquet")
)


In [0]:
users_schema = StructType([
    StructField("Id", LongType(), False),
    StructField("Reputation", IntegerType(), False),
    StructField("CreationDate", TimestampType(), False),
    StructField("DisplayName", StringType(), False),
    StructField("LastAccessDate", TimestampType(), False),
    StructField("WebsiteUrl", StringType(), True),
    StructField("Location", StringType(), True),
    StructField("AboutMe", StringType(), True),
    StructField("Views", LongType(), False),
    StructField("UpVotes", LongType(), False),
    StructField("DownVotes", LongType(), False),
    StructField("AccountId", LongType(), True)
])

users = (
    spark.read
    .format("xml")
    .options(rootTag="xml_split:root", rowTag="row", attributePrefix="")
    .load(f"{data_dir}/Users-*.xml", schema=users_schema)
)

(
  users
  .withColumn("Year", F.year("CreationDate"))
  .write
  # .format("parquet")
  .partitionBy("Year")
  .mode("overwrite")
  .save(f"{data_dir}/users.parquet")
)

In [0]:
votes_schema = StructType([
    StructField("Id", LongType(), False),
    StructField("PostId", LongType(), False),
    StructField("VoteTypeId", ShortType(), False),
    StructField("UserId", LongType(), True),
    StructField("CreationDate", TimestampType(), False),
    StructField("BountyAmount", IntegerType(), True)
])

votes = (
    spark.read
    .format("xml")
    .options(rootTag="xml_split:root", rowTag="row", attributePrefix="")
    .load(f"{data_dir}/Votes-*.xml", schema=votes_schema)
)

(
  votes
  .withColumn("Year", F.year("CreationDate"))
  .write
  # .format("parquet")
  .partitionBy("Year")
  .mode("overwrite")
  .save(f"{data_dir}/votes.parquet")
)

In [0]:
posts_schema = StructType([
    StructField("Id", LongType(), False),
    StructField("PostTypeId", ShortType(), False),
    StructField("AcceptedAnswerId", LongType(), True),
    StructField("ParentId", LongType(), True),
    StructField("CreationDate", TimestampType(), False),
    StructField("Score", IntegerType(), False),
    StructField("ViewCount", IntegerType(), True),
    StructField("Body", StringType(), False),
    StructField("OwnerUserId", LongType(), True),
    StructField("OwnerDisplayName", StringType(), True),
    StructField("LastEditorUserId", LongType(), True),
    StructField("LastEditorDisplayName", StringType(), True),
    StructField("LastEditDate", TimestampType(), True),
    StructField("LastActivityDate", TimestampType(), False),
    StructField("Title", StringType(), True),
    StructField("Tags", StringType(), True),
    StructField("AnswerCount", IntegerType(), True),
    StructField("CommentCount", IntegerType(), True),
    StructField("ClosedDate", TimestampType(), True),
    StructField("CommunityOwnedDate", TimestampType(), True),
    StructField("ContentLicense", StringType(), False)
])

posts = (
    spark.read
    .format("xml")
    .options(rootTag="xml_split:root", rowTag="row", attributePrefix="")
    .load(f"{data_dir}/Posts-*.xml", schema=posts_schema)
)

(
  posts
  .withColumn("Year", F.year("CreationDate"))
  .write
  # .format("parquet")
  .partitionBy("Year")
  .mode("overwrite")
  .save(f"{data_dir}/posts.parquet")
)

In [0]:
%sh

cd /dbfs/tmp

rm -rf \
  tags.parquet/_delta_log \
  post_links.parquet/_delta_log \
  badges.parquet/_delta_log \
  comments.parquet/_delta_log \
  users.parquet/_delta_log \
  votes.parquet/_delta_log \
  posts.parquet/_delta_log